#### Mohamad Quteifan
#### Professor: Amirfarrokh Iranitalab
#### DSC 650: Big Data
#### Assignment 2.1-2.4 (2.5 included in the bottom): Complete the code in kvdb.ipynb, etc. databases
#### Date: 03/28/2021

### Import the data needed for assignments 2.1-2.4

In [8]:
import pandas as pd
import s3fs

s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={
        'endpoint_url': 'https://storage.budsc.midwest-datascience.com'
    }
)

df = pd.read_csv(
    s3.open('data/external/tidynomicon/site.csv', mode='rb')
)

### Assignment 2.1: complete the code in KVDB.ipynb

In [9]:
import json
from pathlib import Path
import os

import pandas as pd
import s3fs


def read_cluster_csv(file_path, endpoint_url='https://storage.budsc.midwest-datascience.com'):
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    return pd.read_csv(s3.open(file_path, mode='rb'))

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

people_json = kv_data_dir.joinpath('people.json')
visited_json = kv_data_dir.joinpath('visited.json')
sites_json = kv_data_dir.joinpath('sites.json')
measurements_json = kv_data_dir.joinpath('measurements.json')



In [10]:
def _load_json(json_path):
    with open(json_path) as f:
        return json.load(f)
    

In [11]:

from pathlib import Path
import json
import os

from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


class DocumentDB(object):
    def __init__(self, db_path):
        ## You can use the code from the previous exmaple if you would like
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')

        self._db_path = Path(db_path)
        self._db = None
        self._person_lookup = _load_json(people_json)
        self._visit_lookup = _load_json(visited_json)
        self._site_lookup = _load_json(sites_json)
        self._measurements_lookup = _load_json(measurements_json)
        self._load_db()
    
    def _get_site(self, site_id):
        return self._site_lookup[str(site_id)]
    
    def _get_measurements(self, person_id):
        measurements = []
        for values in self._measurements_lookup.values():
            measurements.extend([value for value in values if str(['person_id']) == str(person_id)])
        return measurements
    
    def _get_visit(self, visit_id):
        visit = self._visit_lookup.get(str(visit_id))
        site_id = str(visit['site_id'])
        site = self._site_lookup(site_id)
        visit['site'] = site
        return visit
    
    def _load_db(self):
        self._db = TinyDB(self._db_path)
        persons = self._person_lookup.items()
        for person_id, record in persons:
            measurements = self._get_measurements(person_id)
            visit_ids = set([measurement['visit_id'] for measurement in measurements])
            visits = []
            for visit_id in visit_ids:
                visit = self._get_visit(visit_id)
                visit['measurements'] = [
                    measurement for measurement in measurements
                    if visit_id == measurement['visit_id']
                ]
                visits.append(visit)
            record['visits'] = visits
            self._db.insert(record)

In [12]:
class KVDB(object):
    def __init__(self, db_path):
        self._db_path = Path(db_path)
        self._db = {}
        self._load_db()

    def _load_db(self):
        if self._db_path.exists():
            with open(self._db_path) as f:
                self._db = json.load(f)

    def get_value(self, key):
        return self._db.get(key)

    def set_value(self, key, value):
        self._db[key] = value

    def save(self):
        with open(self._db_path, 'w') as f:
            json.dump(self._db, f, indent=2)

In [13]:
def create_sites_kvdb():
    db = KVDB(sites_json)
    df = read_cluster_csv('data/external/tidynomicon/site.csv')
    print(df)
    for site_id, group_df in df.groupby('site_id'):
        db.set_value(site_id, group_df.to_dict(orient='records')[0])
    db.save()

def create_people_kvdb():
    db = KVDB(people_json)
    df = read_cluster_csv('data/external/tidynomicon/person.csv')
    print(df)
    for person_id, group_df in df.groupby('person_id'):
        db.set_value(person_id, group_df.to_dict(orient='records')[0])
    db.save()

def create_visits_kvdb():
    db = KVDB(visited_json)
    df = read_cluster_csv('data/external/tidynomicon/visited.csv')
    print(df)
    for composite_key, group_df in df.groupby(['visit_id', 'site_id']):
        db.set_value(str(composite_key), group_df.to_dict(orient='records')[0])
    db.save()

def create_measurements_kvdb():
    db = KVDB(measurements_json)
    df = read_cluster_csv('data/external/tidynomicon/measurements.csv')
    print(df)
    for composite_key, group_df in df.groupby(['person_id','visit_id', 'quantity']):
        db.set_value(str(composite_key), group_df.to_dict(orient='records')[0])
    db.save()

In [14]:
create_sites_kvdb()
create_people_kvdb()
create_visits_kvdb()
create_measurements_kvdb()

  site_id  latitude  longitude
0    DR-1    -49.85    -128.57
1    DR-3    -47.15    -126.72
2   MSK-4    -48.87    -123.40
  person_id personal_name family_name
0      dyer       William        Dyer
1        pb         Frank     Pabodie
2      lake      Anderson        Lake
3       roe     Valentina     Roerich
4  danforth         Frank    Danforth
   visit_id site_id  visit_date
0       619    DR-1  1927-02-08
1       622    DR-1  1927-02-10
2       734    DR-3  1930-01-07
3       735    DR-3  1930-01-12
4       751    DR-3  1930-02-26
5       752    DR-3         NaN
6       837   MSK-4  1932-01-14
7       844    DR-1  1932-03-22
    visit_id person_id quantity  reading
0        619      dyer      rad     9.82
1        619      dyer      sal     0.13
2        622      dyer      rad     7.80
3        622      dyer      sal     0.09
4        734        pb      rad     8.41
5        734      lake      sal     0.05
6        734        pb     temp   -21.50
7        735        pb      rad 

In [15]:
kvdb_path = 'measurements.json'
kvdb = KVDB(kvdb_path)
key = (619, 'dryer')
value = dict(
    visit_id=619,
    person_id='dyer',
    quantity='rad'
)
kvdb.set_value(key, value)
retrieved_value = kvdb.get_value(key)

In [16]:
kvdb_path = 'visits.json' 
kvdb = KVDB(kvdb_path) 
key = (619, 'DR-1') 
value = dict(
    visit_id=619, 
    site_id='DR-1', 
    visit_date='1927-02-08'
) 
kvdb.set_value(key, value) 
retrieved_value = kvdb.get_value(key)

### Assignment 2.2: Now we will create a simple document database using the tinydb library

In [17]:
def _load_json(json_path):
    with open(json_path) as f:
        return json.load(f)

In [20]:
from pathlib import Path
import json
import os
from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

def _load_json(json_path):
    with open(json_path) as f:
        return json.load(f)

class DocumentDB(object):
    def __init__(self, db_path):
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')
        self._db_path = Path(db_path)
        self._db = None
        self._person_lookup = _load_json(people_json)
        self._visit_lookup = _load_json(visited_json)
        self._site_lookup = _load_json(sites_json)
        self._measurements_lookup = _load_json(measurements_json)

        self._load_db()
        
    def _get_site(self, site_id):
        return self._site_lookup[str(site_id)]
    def _get_measurements(self, person_id):
        measurements = []
        for values in self._measurements_lookup.values():
            measurements.extend([value for value in values if str(['person_id']) == str(person_id)])
        return measurements
    def _get_visit(self, visit_id):
        visit = self._visit_lookup.get(str(visit_id))
        site_id = str(visit['site_id'])
        site = self._site_lookup(site_id)
        visit['site'] = site
        return visit
    def _load_db(self):
        self._db = TinyDB(self._db_path)
        persons = self._person_lookup.items()
        for person_id, record in persons:
            measurements = self._get_measurements(person_id)
            visit_ids = set([measurement['visit_id'] for measurement in measurements])
            visits = []
            for visit_id in visit_ids:
                visit = self._get_visit(visit_id)
                visit['measurements'] = [
                    measurement for measurement in measurements
                    if visit_id == measurement['visit_id']
                ]
                visits.append(visit)
            record['visits'] = visits
            self._db.insert(record)

In [21]:
db_path = results_dir.joinpath('patient_info.json')
if db_path.exists():
    os.remove(db_path)

db = DocumentDB(db_path)

### 2.3 & 2.4: In this part, you will create a SQLite database

In [22]:
from pathlib import Path
import os
import sqlite3

import s3fs
import pandas as pd

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


def read_cluster_csv(file_path, endpoint_url='https://storage.budsc.midwest-datascience.com'):
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    return pd.read_csv(s3.open(file_path, mode='rb'))

#### Create and Load Measurements Table

In [23]:
def create_measurements_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS measurements (
        visit_id integer NOT NULL,
        person_id text NOT NULL,
        quantity text,
        reading real,
        FOREIGN KEY (visit_id) REFERENCES visits (visit_id),
        FOREIGN KEY (person_id) REFERENCES people (people_id)
        );
    """

    c = conn.cursor()
    c.execute(sql)
    
def load_measurements_table(conn):
    create_measurements_table(conn)
    df = read_cluster_csv('data/external/tidynomicon/measurements.csv')
    measurements = df.values
    c = conn.cursor()
    c.execute('DELETE FROM measurements;') # Delete data if exists
    c.executemany('INSERT INTO measurements VALUES (?,?,?,?)', measurements)

#### Create and Load People Table

In [24]:
def create_people_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS people (
        people_id text NOT NULL,
        personal_name text,
        family_name text
        );
    """
    c = conn.cursor()
    c.execute(sql)
    
def load_people_table(conn):
    create_people_table(conn)
    df_people = read_cluster_csv('data/external/tidynomicon/person.csv')
    people = df_people.values
    c = conn.cursor()
    c.execute('DELETE FROM people;') # Delete data if exists
    c.executemany('INSERT INTO people VALUES (?,?,?)', people)

#### Create and Load Sites Table

In [25]:
def create_sites_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id text PRIMARY KEY,
        latitude double NOT NULL,
        longitude double NOT NULL
        );
    """

    c = conn.cursor()
    c.execute(sql)

def load_sites_table(conn):
    create_sites_table(conn)
    df_sites = read_cluster_csv('data/external/tidynomicon/site.csv')
    sites = df_sites.values
    c = conn.cursor()
    c.execute('DELETE FROM sites;') # Delete data if exists
    c.executemany('INSERT INTO sites VALUES (?,?,?)', sites)

#### Create and Load Visits Table

In [26]:
def create_visits_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS visits (
        visit_id integer PRIMARY KEY,
        site_id text NOT NULL,
        visit_date text,
        FOREIGN KEY (site_id) REFERENCES sites (site_id)
        );
    """

    c = conn.cursor()
    c.execute(sql)

def load_visits_table(conn):
    create_visits_table(conn)
    df_visited = read_cluster_csv('data/external/tidynomicon/visited.csv')
    visits = df_visited.values
    c = conn.cursor()
    c.execute('DELETE FROM visits;') # Delete data if exists
    c.executemany('INSERT INTO visits VALUES (?,?,?)', visits)

#### Create DB and Load Tables

In [27]:
db_path = results_dir.joinpath('patient-info.db')
conn = sqlite3.connect(str(db_path))

load_people_table(conn) 
load_sites_table(conn)
load_visits_table(conn)
load_measurements_table(conn)

conn.commit()
conn.close()



### Assignment 2.5: Go to the Wikidata Query Service website and perform the following SPARQL query.

CSV(COMMA SEPERATED VALUE

event,         eventLabel,   date

1. wd:Q10377272, TV Kids, 15 March 2021
2. wd:Q29325343, 2021 Western Australian state election, 13 March 2021
3. wd:Q56401728, 2021 FC Barcelona presidential election, 7 March 2021
4. wd:Q56401728, 2021 FC Barcelona presidential election, 7 March 2021
5. wd:Q85772546, Juno Awards of 2021, 28 March 2021
6. wd:Q97164943, The Flash, season 7, 2 March 2021
7. wd:Q97579469, 35th Goya Awards, 6 March 2021
8. wd:Q101466553, The European Data Conference on Reference Data and Semantics,    16 March 2021
9. wd:Q102046823, Koh-Lanta, season 22, 12 March 2021
10. wd:Q104099653, 2020–21 Biathlon World Cup – Stage 8, 4 March 2021
